In [1]:
import pandas as pd

from Project.Database import Db
from optimisation_problem import hourly_house_df, load_app_stats, slice_emission_vector, power_consumption_vector, NZERTF_optimiser, SE_time_df, json_to_dataframe

In [2]:
def find_emissions(df, emission_vec, day_number):
    energy_vec = df.loc[lambda self: self['Day'] == day_number, 'Consumption'].reset_index(drop=True)
    energy_consumed = energy_vec.multiply(emission_vec)
    for index, emission in energy_consumed[lambda self: self > 0].iteritems():
        df.loc[lambda self: (self['Day'] == day) & (self['Hour'] == index), 'Emission'] += emission
    return df

In [ ]:
meta = Db.load_data(meta=True, hourly=False, year=2, consumption=False).loc[
    lambda self: (~self['Consumer_Match'].isna()), 'Consumer_Match']

movable_appliances = ['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum', 'Load_StatusClothesWasher',
                      'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']

appliances = ['Timestamp'] + meta.index.tolist() + meta.tolist()

patterns = SE_time_df(json_to_dataframe(
    year=2,
    level=1,
    exclude_follows=True,
    with_redundancy=False).loc[lambda self: self['pattern'].isin(movable_appliances)])

power_consumption = power_consumption_vector(movable_appliances=movable_appliances)

# with redundancy <- w.r
# without redundancy <- w.o.r
# movable appliances unoptimised <- m.a.u.o
# movable appliances optimised <- m.a.o
NZERTF_optimisation = {
    'w.r': Db.load_data(year=2, hourly=False)[appliances],
    'w.o.r': Db.load_data(year=2, hourly=False, with_redundancy=False)[appliances]
}

NZERTF_optimisation.update({
    'm.a.u.o': NZERTF_optimisation['w.o.r'].copy()[['Timestamp'] + movable_appliances]
})

timestamps = pd.to_datetime(
    NZERTF_optimisation['w.r']['Timestamp'].dt.strftime('%Y-%m-%d %H').unique(),
    format='%Y-%m-%d %H')

production = Db.load_data(
    consumption=False,
    production=True,
    year=2)['CO2(Grams)/kWh'][lambda self: self.index.isin(timestamps)]
production = production.groupby(pd.to_datetime(production.index.strftime('%Y-%m-%d %H'))).sum()

for key in NZERTF_optimisation.keys():
    if all([att in NZERTF_optimisation[key].columns for att in meta.tolist()]):
        NZERTF_optimisation.update({
            key: pd.concat(
                objs=(hourly_house_df(house_df=NZERTF_optimisation[key][['Timestamp'] + meta.index.tolist()].copy(),
                                      aggregate_func='mean'),
                      hourly_house_df(house_df=NZERTF_optimisation[key][['Timestamp'] + meta.tolist()].copy(),
                                      aggregate_func='sum')[meta.tolist()]),
                axis=1)
        })
        NZERTF_optimisation[key]['Consumption'] = NZERTF_optimisation[key][meta.tolist()].sum(1).div(60_000)
        NZERTF_optimisation[key].drop(labels=meta.tolist(), inplace=True, axis=1)
    else:
        # print(NZERTF_optimisation[key].loc[(lambda self: (self[movable_appliances] != 0) |
        #                                                  (self[movable_appliances] != 1)), movable_appliances])
        NZERTF_optimisation.update({
            key: hourly_house_df(house_df=NZERTF_optimisation[key], aggregate_func='max')
        })

        for app in movable_appliances:
            timespan = patterns[app]['Timespan'][0]
            indexes = NZERTF_optimisation[key][app].loc[
                lambda self: (self != self.shift(1)) & (self == 1)].index.tolist()
            NZERTF_optimisation[key][app].loc[lambda self: (~self.index.isin(indexes))] = 0
            for index in indexes:
                remaining = timespan
                hour = index
                while remaining > 0:
                    NZERTF_optimisation[key][app][hour] = min(remaining, 1)
                    remaining = max(remaining - 1, 0)
                    hour += 1

        NZERTF_optimisation[key]['Consumption'] = NZERTF_optimisation[key][movable_appliances].dot(
            power_consumption).div(1_000)

    for day in NZERTF_optimisation[key]['Day'].unique():
        NZERTF_optimisation[key] = find_emissions(
            df=NZERTF_optimisation[key],
            emission_vec=slice_emission_vector(
                production_vectors=production,
                day_number=day), day_number=day)

NZERTF_optimisation.update({
    'm.a.o': NZERTF_optimiser()
})

NZERTF_emission = {}

for key in NZERTF_optimisation.keys():
    NZERTF_emission.update({
        key: NZERTF_optimisation[key]['Emission'].sum()
    })

/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_73307/1592343466.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NZERTF_optimisation[key][app].loc[lambda self: (~self.index.isin(indexes))] = 0
/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_73307/1592343466.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NZERTF_optimisation[key][app][hour] = min(remaining, 1)
/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_73307/1592343466.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [ ]:
for key, value in NZERTF_optimisation.items():
    print(f'{key}: {value["Emission"]}')

In [ ]:
NZERTF_emission

In [ ]:
redundancy_reduction = NZERTF_emission['w.r'] - NZERTF_emission['w.o.r']
optimisation_reduction = NZERTF_emission['m.a.u.o'] - NZERTF_emission['m.a.o']
total_reduction = redundancy_reduction + optimisation_reduction

In [ ]:
redundancy_reduction

In [ ]:
optimisation_reduction

In [ ]:
total_reduction